## Loan Recommender

In [17]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import ast

In [18]:
# Initialize the LLM (using Ollama in this example).
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [19]:

# Create a prompt template for extracting parameters.
extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).\n"
        "- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).\n"
        "- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).\n"
        "- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).\n"
        "- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).\n"
        "- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).\n\n"
        "If a parameter is not mentioned in the input, set its value to None.\n\n"
        "Example output:\n"
        "{{\n"
        '  "deposit_amount": 5000000,\n'
        '  "deposit_duration": 6,\n'
        '  "loan_amount": "None",\n'
        '  "credit_score": "None",\n'
        '  "number_of_installments": "None",\n'
        '  "interest_rate": "None"\n'
        "}}"
    )
)
# Create the extraction chain.
extraction_chain = LLMChain(llm=llm, prompt=extraction_prompt, verbose=True)

#

In [20]:

# Define the Function to Parse the LLM Output

def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the dictionary from the LLM output.
    First, find the substring between the first '{' and the last '}'.
    Then try to parse that substring as JSON.
    If that fails (for example, due to single quotes), fall back to ast.literal_eval.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        try:
            return ast.literal_eval(json_str)
        except Exception as e2:
            raise ValueError(f"Error parsing extraction result: {e2}")


In [21]:


# Example user input.
# user_input = (
#     "I deposited 5 million Tomans for 6 months. Also, I took a loan of 250000000 from Bank X. "
#     "My credit score is A. I have 12 installments "
# )


user_input = (
    "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۹ ماهه پرداختش کنم"
)

# user_input = (
#     "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم تعداد ۸ قسط پرداختش کنم"
# )



# and the interest rate is 4.
# Run the extraction chain.
extraction_result = extraction_chain.run({"user_input": user_input})
print("Extraction result:")
print(extraction_result)

# Function to parse the LLM output into a Python dictionary.
def parse_output(result_str: str) -> dict:
    # Find the first '{' and the last '}' in the output.
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception:
        # Fallback to ast.literal_eval if JSON parsing fails.
        return ast.literal_eval(json_str)

# Parse the extracted output.
parsed_params = parse_output(extraction_result)
print("Parsed parameters:")
print(parsed_params)




> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input:
"من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۹ ماهه پرداختش کنم"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).
- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).
- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).
- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).
- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).
- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).

If a parameter is not mentioned in the input, set its value to None.

Example output:
{
  "deposit_amount": 5000000,
  "deposit_duration": 6,
  "loan_amount": "None",
  "credit_score": "

## Check validation of parameters' value 

In [22]:
def get_optional_parameters(extracted_params: dict) -> dict:
    """
    Prompt the user to verify or provide values for each parameter.
    The user can choose to leave a parameter blank (resulting in a value of None).
    
    Valid criteria and hints:
      - deposit_amount: Any number (no limitations) or None.
      - deposit_duration: An integer between 1 and 12 (months) or None.
      - loan_amount: An integer up to 300000000 or None.
      - credit_score: One of the following strings: A, B, C, D, E, None.
      - number_of_installments: One of: 4, 6, 9, 12, 18, 24, 36, 48 or None.
      - interest_rate: One of: 0, 2, 4, 18, 23 or None.
    """
    # Define valid criteria (as lambda functions) for each parameter.
    valid_criteria = {
        "deposit_amount": lambda x: True,
        "deposit_duration": lambda x: 1 <= x <= 12,
        "loan_amount": lambda x: x <= 300000000,
        "credit_score": lambda x: x in ["A", "B", "C", "D", "E", "None"],
        "number_of_installments": lambda x: x in [4, 6, 9, 12, 18, 24, 36, 48],
        "interest_rate": lambda x: x in [0, 2, 4, 18, 23]
    }
    
    # Hints to display to the user.
    hints = {
        "deposit_amount": "Enter any number (no limitations).",
        "deposit_duration": "Enter an integer between 1 and 12 (months).",
        "loan_amount": "Enter an integer no more than 300000000.",
        "credit_score": "Enter one of: A, B, C, D, E, None.",
        "number_of_installments": "Allowed values: 4, 6, 9, 12, 18, 24, 36, 48.",
        "interest_rate": "Allowed values: 0, 2, 4, 18, 23."
    }
    
    # Iterate over each parameter in the valid_criteria.
    for param, criteria in valid_criteria.items():
        current_val = extracted_params.get(param)
        while True:
            if current_val is not None:
                # Ask the user if they want to keep the extracted value or override it.
                user_input = input(
                    f"The extracted value for '{param}' is {current_val}. Press Enter to keep it, or enter a new value (or leave blank to remove): "
                )
                if user_input.strip() == "":
                    # If the user presses Enter, we keep the current value.
                    candidate = current_val
                else:
                    candidate = user_input.strip()
            else:
                # No value was extracted, so ask for one (user can leave blank to remain None).
                candidate = input(f"Please enter a value for '{param}' (or press Enter to leave it blank): ").strip()
                if candidate == "":
                    # Leave as None.
                    extracted_params[param] = None
                    break
            
            try:
                # Convert the candidate to the correct type.
                if param in ["deposit_duration", "loan_amount", "number_of_installments", "interest_rate"]:
                    candidate_val = int(candidate)
                elif param == "deposit_amount":
                    candidate_val = float(candidate)
                else:
                    # For credit_score, treat it as a string and convert to uppercase.
                    candidate_val = candidate.upper()
                    
                # Validate the candidate value using the defined criteria.
                if not criteria(candidate_val):
                    print(f"Invalid value for {param}. {hints[param]}")
                    current_val = None
                    continue
                
                # If valid, store it and break out of the loop.
                extracted_params[param] = candidate_val
                break
            except ValueError:
                print(f"Could not convert the input for {param} to the required type. {hints[param]}")
                current_val = None
                
    return extracted_params


In [23]:
# Example user input (this could be from a chatbot, web form, etc.).
# user_input = (
#     "I deposited 5 million Tomans for 6 months. I took a loan of 250000000 from Bank X. "
#     "My credit score is A. I have 12 installments and the interest rate is 4."
# )

user_input = (
    "من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۸ ماهه پرداختش کنم"
)


# Use the LLM extraction chain to get parameters from the input.
extraction_result = extraction_chain.run({"user_input": user_input})
print("LLM Extraction Result:")
print(extraction_result)

# Parse the extraction result to a Python dictionary.
extracted_params = parse_extraction_result(extraction_result)
print("Extracted Parameters (from LLM):")
print(extracted_params)

# Validate or prompt the user for any missing/incorrect parameters.
final_params = get_optional_parameters(extracted_params)
print("Final Validated Parameters:")
print(final_params)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input:
"من میخوام یه وام ۳۵ میلیونی بگیرم. کلا هم میتونم توی اقساط ۸ ماهه پرداختش کنم"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited (if mentioned; otherwise, None).
- deposit_duration: The duration of the deposit in months (should be between 1 and 12; if mentioned; otherwise, None).
- loan_amount: The total loan amount (if mentioned; maximum allowed is 300000000; otherwise, None).
- credit_score: One of the following values: A, B, C, D, E, None (if mentioned; otherwise, None).
- number_of_installments: Allowed values are 4, 6, 9, 12, 18, 24, 36, 48 (if mentioned; otherwise, None).
- interest_rate: Allowed values are 0, 2, 4, 18, 23 (if mentioned; otherwise, None).

If a parameter is not mentioned in the input, set its value to None.

Example output:
{
  "deposit_amount": 5000000,
  "deposit_duration": 6,
  "loan_amount": "None",
  "credit_score": "